<a href="https://colab.research.google.com/github/anup-66/MACHINE-LEARNING-PROJECTS/blob/master/Abstractive_text_summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
import re
import pickle
import string
import unicodedata
from random import randint

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from wordcloud import STOPWORDS, WordCloud

from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras import Input, Model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.layers import LSTM, Bidirectional, Dense, Embedding, TimeDistributed

In [9]:
!pip install -q contractions==0.0.48


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.1/110.1 KB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.5/287.5 KB 11.9 MB/s eta 0:00:00


In [3]:
tpu = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)

# instantiate a distribution strategy
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

In [4]:
filename1 = '/content/news_summary.csv'
filename2 = '/content/news_summary_more.csv'

df1 = pd.read_csv(filename1, encoding='iso-8859-1').reset_index(drop=True)
df2 = pd.read_csv(filename2, encoding='iso-8859-1').reset_index(drop=True)

In [5]:
df1.sample(5)

,author,date,headlines,read_more,text,ctext
3906,Rini Sinha,"07 Apr 2017,Friday",Thought it was late April fool's joke: Akshay ...,http://indiatoday.intoday.in/story/akshay-kuma...,Actor Akshay Kumar has said that when he got t...,He has given innumerable hits and some great p...
2805,Ayushi Ahluwalia,"29 Jun 2017,Thursday",Punjab HC stays govt policy on liquor sale at ...,http://indiatoday.intoday.in/story/serving-liq...,The Punjab and Haryana High Court on Wednesday...,"Chandigarh, June 28 (PTI) The Punjab and Harya..."
1664,Vaibhav Gupta,"09 Jan 2017,Monday",BJP leader moves EC against Mayawati for 'cast...,http://indiatoday.intoday.in/story/bjp-leader-...,BJP leader Neeraj Saxena has moved the Electio...,"Lucknow, Jan 9 (PTI) A BJP leader has moved th..."
1079,Tarun Khanna,"20 Feb 2017,Monday",Mopeds on discount for women: TN?s new CM?s 1s...,http://indiatoday.intoday.in/story/tamil-nadu-...,Within days after proving the majority of his ...,Issuing his first set of orders after having s...
4423,Mansha Mahajan,"01 Mar 2017,Wednesday","Need regulatory body for medical devices, says...",http://indiatoday.intoday.in/story/india-needs...,Biocon Chairperson Kiran Mazumdar-Shaw has cal...,NaN


In [6]:
df1_columns = df1.columns.tolist()
df1_columns.remove('headlines')
df1_columns.remove('text')
df1.drop(df1_columns, axis='columns', inplace=True)

df = pd.concat([df1, df2], axis='rows')
del df1, df2

# Shuffling the df
df = df.sample(frac=1).reset_index(drop=True)

print(f'Dataset size: {len(df)}')
df.sample(5)

Dataset size: 102915


,headlines,text
7343,Amitabh is more tech-savvy than Aamir: Fatima ...,"Fatima Sana Shaikh, who has worked with Amitab..."
50729,US court orders Trump admin to reinstate 'Drea...,A US federal judge on Friday ordered President...
9798,Dhoni completes 800 dismissals in internationa...,Indian wicketkeeper MS Dhoni completed 800 dis...
75868,Vatican uses term 'LGBT' in official document ...,"The Vatican has used the abbreviation 'LGBT', ..."
91571,Death toll due to collapse of Mumbai building ...,The number of people killed when a multi-store...


Here The headline will be seen as a sumary to compare generated summary.

In [10]:
"""Date preparation codes """
from contractions import contractions_dict
def expand_contractions(text, contraction_map=contractions_dict):
    # Using regex for getting all contracted words
    contractions_keys = '|'.join(contraction_map.keys())
    contractions_pattern = re.compile(f'({contractions_keys})', flags=re.DOTALL)

    def expand_match(contraction):
        # Getting entire matched sub-string
        match = contraction.group(0)
        expanded_contraction = contraction_map.get(match)
        if not expand_contractions:
            print(match)
            return match
        return expanded_contraction

    expanded_text = contractions_pattern.sub(expand_match, text)
    expanded_text = re.sub("'", "", expanded_text)
    return expanded_text

expand_contractions("i'd think y'all can do this.")

'id think you all can do this.'

In [11]:
# converting all to lower case 

df.text = df.text.apply(str.lower)
df.headlines = df.headlines.apply(str.lower)

In [12]:
df.headlines = df.headlines.apply(expand_contractions)
df.text = df.text.apply(expand_contractions)
df.sample(4)

,headlines,text
4262,scientists make 1st ever intercontinental quan...,austrian and chinese acadthemies of sciences h...
76846,brazil sends in army as protesting truck drive...,brazils president michel tthemer has ordered t...
45835,"rishi an accomplished artiste, working with hi...","amitabh bachchan has said rishi kapoor is an ""..."
98871,"afghanistan launches t20 league, wants indian ...",the afghanistan cricket board reportedly wants...


In [24]:
# Remove puncuation from word
# !pip install cleantext==1.1.1
from cleantext import clean
def rm_punc_from_word(word):
    clean_alphabet_list = [
        alphabet for alphabet in word if alphabet not in string.punctuation
    ]
    return ''.join(clean_alphabet_list)

print(rm_punc_from_word('#cool!'))


# Remove puncuation from text
def rm_punc_from_text(text):
    
    clean_word_list = [rm_punc_from_word(word) for word in text]
    return ''.join(clean_word_list)

print(rm_punc_from_text("hello , how are you?"))

cool
hello  how are you


In [25]:
def rm_number_from_text(text):
    text = re.sub('[0-9]+', '', text)
    return ' '.join(text.split())  # to rm `extra` white space


In [27]:
# Remove stopwords from text
def rm_stopwords_from_text(text):
    _stopwords = stopwords.words('english')
    text = text.split()
    word_list = [word for word in text if word not in _stopwords]
    return ' '.join(word_list)

In [28]:
def clean_text(text):
    text = text.lower()
    text = rm_punc_from_text(text)
    text = rm_number_from_text(text)
    text = rm_stopwords_from_text(text)

    # there are hyphen(–) in many titles, so replacing it with empty str
    # this hyphen(–) is different from normal hyphen(-)
    text = re.sub('–', '', text)
    text = ' '.join(text.split())  # removing `extra` white spaces

    # Removing unnecessary characters from text
    text = re.sub("(\\t)", ' ', str(text)).lower()
    text = re.sub("(\\r)", ' ', str(text)).lower()
    text = re.sub("(\\n)", ' ', str(text)).lower()
    # remove accented chars ('Sómě Áccěntěd těxt' => 'Some Accented text')
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode(
        'utf-8', 'ignore'
    )

    text = re.sub("(__+)", ' ', str(text)).lower()
    text = re.sub("(--+)", ' ', str(text)).lower()
    text = re.sub("(~~+)", ' ', str(text)).lower()
    text = re.sub("(\+\++)", ' ', str(text)).lower()
    text = re.sub("(\.\.+)", ' ', str(text)).lower()

    text = re.sub(r"[<>()|&©ø\[\]\'\",;?~*!]", ' ', str(text)).lower()

    text = re.sub("(mailto:)", ' ', str(text)).lower()
    text = re.sub(r"(\\x9\d)", ' ', str(text)).lower()
    text = re.sub("([iI][nN][cC]\d+)", 'INC_NUM', str(text)).lower()
    text = re.sub("([cC][mM]\d+)|([cC][hH][gG]\d+)", 'CM_NUM',
                  str(text)).lower()
    text = re.sub("(\.\s+)", ' ', str(text)).lower()
    text = re.sub("(\-\s+)", ' ', str(text)).lower()
    text = re.sub("(\:\s+)", ' ', str(text)).lower()
    text = re.sub("(\s+.\s+)", ' ', str(text)).lower()

    try:
        url = re.search(r'((https*:\/*)([^\/\s]+))(.[^\s]+)', str(text))
        repl_url = url.group(3)
        text = re.sub(r'((https*:\/*)([^\/\s]+))(.[^\s]+)', repl_url, str(text))
    except Exception as e:
        pass

    text = re.sub("(\s+)", ' ', str(text)).lower()
    text = re.sub("(\s+.\s+)", ' ', str(text)).lower()

    return text

clean_text("Mrs. Robinson, 343 -- you're trying to fool &^%me, aren't you?")
    

'mrs robinson youre trying fool arent'

In [29]:
df.text = df.text.apply(clean_text)
df.headlines = df.headlines.apply(clean_text)

In [31]:
df.to_csv('/content/cleaned_data.csv')

Adding tokens in the start and the end to let the learning , model know where to start and where to end.

In [32]:
start_token = 'sostok'
end_token = 'eostok'
df.headlines = df.headlines.apply(lambda x: f'{start_token} {x} {end_token}')

In [33]:
text_count = [len(sentence.split()) for sentence in df.text]
headlines_count = [len(sentence.split()) for sentence in df.headlines]

pd.DataFrame({'text': text_count, 'headlines': headlines_count})

,text,headlines
0,35,9
1,33,8
2,33,8
3,36,9
4,31,10
...,...,...
102910,29,8
102911,31,8
102912,30,11
102913,34,7


In [34]:
max_text_len = 2000
max_summary_len = 20

# select the summary and text between their defined max lens respectively
def trim_text_and_summary(df, max_text_len, max_summary_len):
    cleaned_text = np.array(df['text'])
    cleaned_summary = np.array(df['headlines'])

    short_text = []
    short_summary = []

    for i in range(len(cleaned_text)):
        if len(cleaned_text[i].split()) <= max_text_len and len(
            cleaned_summary[i].split()
        ) <= max_summary_len:
            short_text.append(cleaned_text[i])
            short_summary.append(cleaned_summary[i])

    df = pd.DataFrame({'text': short_text, 'summary': short_summary})
    return df
  
df = trim_text_and_summary(df, max_text_len, max_summary_len)
print(f'Dataset size: {len(df)}')


Dataset size: 102915
